## Install the following libraries, if not done already, before running the below script

#### !pip install tensorflow
#### !pip install scikit-learn
#### !pip install numpy 
#### !pip install pandas 
#### !pip install pyDOE 
#### !pip install scipy
#### !pip install matplotlib
#### !pip install pypickle

In [ ]:
import numpy as np
import pandas as pd
from pyDOE import lhs  # Latin Hypercube Sampling
from scipy.optimize import newton
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from matplotlib import pyplot as plt
import os
from sklearn.model_selection import GroupShuffleSplit
import pickle

# Ensure reproducibility
np.random.seed(42)
tf.random.set_random_seed(42)


# Material constant ranges from Table 1
material_constant_ranges = {
    "E": [191000e6, 195000e6],  # Young's modulus in MPa
    "sigma_y": [90e6, 120e6],  # Yield stress in MPa
    "h": [5,10],  # Isotropic hardening rate
}
# Generate material constants using Latin Hypercube Sampling
def generate_material_constants_lhs(num_samples, ranges):
    num_variables = len(ranges)
    lhs_samples = lhs(num_variables, samples=num_samples)
    material_constants = []

    for i, (key, (low, high)) in enumerate(ranges.items()):
        samples = lhs_samples[:, i] * (high - low) + low
        material_constants.append(samples)

    return np.column_stack(material_constants)


# Generate strain history
def generate_strain_history(num_samples, lb, ub):
    strain_history = np.zeros(num_samples)
    cumulative_strain = 0  # Start with zero cumulative strain

    # First range: Accumulate small positive strain increments
    for i in range(300):
        increment = 0.0001
        cumulative_strain += increment
        strain_history[i] = cumulative_strain

    # Second range: Accumulate negative strain increments
    for i in range(300, 900):
        increment = 0.0001
        cumulative_strain -= increment
        strain_history[i] = cumulative_strain

    # Third range: Return to positive strain increments
    for i in range(900, num_samples):
        increment = 0.0001
        cumulative_strain += increment
        strain_history[i] = cumulative_strain

    return strain_history


# Hooke's law (elastic predictor)
def elastic_predictor(eps, eps_p, E, h, alpha, sigma_y):
    sigma_trial = E * (eps - eps_p)  # Trial stress
    yield_function = np.abs(sigma_trial) - (sigma_y + h * alpha)
    return sigma_trial, yield_function


# Von Mises return mapping algorithm
def von_mises(eps, eps_p, sigma_trial, yield_function, alpha, h, E, sigma_y):
    if yield_function <= 0:
        # Elastic step
        sigma_updated = sigma_trial
        eps_p = eps_p
        alpha = alpha
        delta_gamma=0
        sign = sigma_trial/np.abs(sigma_trial)
    else:
        # Plastic step
        delta_gamma = yield_function / (E + h)
        eps_p = eps_p + delta_gamma* (sigma_trial/np.abs(sigma_trial))
        alpha = alpha + delta_gamma
        #sigma_updated = sigma_trial - E * delta_gamma
        sigma_updated = E*(eps-eps_p)
        sign = sigma_trial/np.abs(sigma_trial)

    return sigma_updated, eps_p, alpha, delta_gamma, sign



# Generate dataset
def generate_dataset(num_samples,num_strain_samples,lb, ub,ranges):
    
    dataset = []
    material_constants = generate_material_constants_lhs(num_samples, ranges)
    #print(material_constants)
    
    for constants in material_constants:
        
        E, sigma_y, h = constants
        strain_history = generate_strain_history(num_strain_samples, lb, ub)

        # Initialize state variables
        alpha, eps_p = 0, 0
    
        for eps in strain_history:
            # Elastic predictor
            sigma_trial, yield_function = elastic_predictor(eps, eps_p, E, h, alpha, sigma_y)

            # Return mapping algorithm
            sigma_updated, eps_p, alpha, delta_gamma, sign  = von_mises(eps, eps_p, sigma_trial, yield_function, alpha, h, E, sigma_y)

            # Store the data
            dataset.append([
                E,sigma_y,h, yield_function, eps, sigma_updated, eps_p, alpha, sign, delta_gamma
            ])

    return np.array(dataset)


# Save the dataset to a CSV file
def save_dataset_to_csv(dataset, filename="generated_dataset_von_mises_training_07_01_2025.csv"):
    columns = ["E","sigma_y","h", "yield_function", "strain", "sigma_updated", "cummulative eps_p", "alpha",'sign', 'plastic_strain']
    df = pd.DataFrame(dataset, columns=columns)
    df.to_csv(filename, index=False)
    print(f"Dataset saved to {filename}")


In [2]:
def normalize_dataset(data, scaler=None):
    if scaler is None:
        scaler = MinMaxScaler()
        normalized_data = scaler.fit_transform(data)
    else:
        normalized_data = scaler.transform(data)
    return normalized_data, scaler

In [3]:
# Build the neural network
def build_model(input_dim):
    model = Sequential([
        Dense(10, activation='sigmoid', input_shape=(input_dim,)),  # Use 10 neurons, sigmoid activation
        Dense(1, activation='linear')     # Linear activation for the output
    ])
    custom_adam = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(optimizer=custom_adam, loss='mean_squared_error', metrics=['mae'])
    return model

In [4]:
# Plot training and validation loss
def plot_loss(history, filename='loss_curve.png'):
    plt.figure(figsize=(8, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss (MSE)')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    #plt.savefig(filename)  # Save the plot as an image file
    #plt.close()  # Close the plot to avoid displaying it in the notebook
    #print(f"Loss curve saved as {filename}")

# Plot predicted vs original values
def plot_predicted_vs_actual(y_test, y_pred, filename='predicted_vs_actual.png'):
    plt.figure(figsize=(8, 6))
    plt.plot(y_test, label='Original Values', color='blue')
    plt.plot(y_pred, label='Predicted Values', color='red', linestyle='--')
    plt.xlabel('Sample index')
    plt.ylabel('Plastic Strain')
    plt.title('Predicted vs. Original Plastic Strain')
    plt.legend()
    plt.grid(True)
    plt.show()
    plt.savefig('predicted_vs_actual')  # Save the plot as an image file
    plt.close()  # Close the plot to avoid displaying it in the notebookLHSLHS
    print(f"Plot saved as {'predicted_vs_actual'}")

In [5]:
training = generate_dataset(num_samples=500, num_strain_samples=1500, lb=0.0001,ub=0.0002, ranges=material_constant_ranges)
#test1 = generate_dataset(lb=0,ub=0.0001, num_strain_samples=1500)
#test2 = generate_dataset(lb=0.0002,ub=0.0004, num_strain_samples=1500)

In [6]:
columns = ["E","sigma_y","h", "yield_function", "strain", "sigma_updated", "cumulative_plastic_strain", "alpha", 'sign','plastic_strain']
df = pd.DataFrame(training, columns=columns)

In [7]:
df.tail(50)

,E,sigma_y,h,yield_function,strain,sigma_updated,cumulative_plastic_strain,alpha,sign,plastic_strain
749950,1.934270e+11,9.676222e+07,5.45703,1.934270e+07,0.0251,9.676222e+07,0.0246,0.142599,1.0,0.0001
749951,1.934270e+11,9.676222e+07,5.45703,1.934270e+07,0.0252,9.676222e+07,0.0247,0.142699,1.0,0.0001
749952,1.934270e+11,9.676222e+07,5.45703,1.934270e+07,0.0253,9.676222e+07,0.0248,0.142799,1.0,0.0001
749953,1.934270e+11,9.676222e+07,5.45703,1.934270e+07,0.0254,9.676222e+07,0.0249,0.142899,1.0,0.0001
749954,1.934270e+11,9.676222e+07,5.45703,1.934270e+07,0.0255,9.676222e+07,0.0250,0.142999,1.0,0.0001
749955,1.934270e+11,9.676222e+07,5.45703,1.934270e+07,0.0256,9.676222e+07,0.0251,0.143099,1.0,0.0001
749956,1.934270e+11,9.676222e+07,5.45703,1.934270e+07,0.0257,9.676222e+07,0.0252,0.143199,1.0,0.0001
749957,1.934270e+11,9.676222e+07,5.45703,1.934270e+07,0.0258,9.676222e+07,0.0253,0.143299,1.0,0.0001
749958,1.934270e+11,9.676222e+07,5.45703,1.934270e+07,0.0259,9.676222e+07,0.0254,0.143399,1.0,0.0001
749959,1.934270e+11,9.676222e+07,5.45703,1.934270e+07,0.0260,9.676222e+07,0.0255,0.143499,1.0,0.0001


In [8]:
save_dataset_to_csv(training)

Dataset saved to generated_dataset_von_mises_training_07_01_2025.csv


In [9]:
df.shape

(750000, 10)

In [10]:
# Separate features and target
X = training[:, :4]  # Features (E, sigma_y, c1, c2, c3, gamma1, gamma2, gamma3, b, Q, strain)
y = training[:, -1]   # Targets (plastic_strain)

In [11]:
training[:, -1] 

array([0.e+00, 0.e+00, 0.e+00, ..., 1.e-04, 1.e-04, 1.e-04])

In [12]:
X_normalized, scaler_X= normalize_dataset(X)
y = y.reshape(-1, 1)
y_normalized, scaler_y= normalize_dataset(y)

In [13]:
# Build the neural network model
print("Building the neural network model...")
model = build_model(X.shape[1])

# Define the early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',         # Monitor the validation loss
    patience=10,                # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True   # Restore model weights from the epoch with the best value of the monitored quantity
)

# Train the model with early stopping
print("Training the model...")
history = model.fit(
    X_normalized, 
    y_normalized, 
    epochs=3000,               # Set a large number of epochs
    batch_size=1000,
    validation_split=0.2, 
    callbacks=[early_stopping],  # Include the early stopping callback
    verbose=1
)

Building the neural network model...
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Training the model...
Train on 600000 samples, validate on 150000 samples
Epoch 1/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 0.0396 - mean_absolute_error: 0.1006 - val_loss: 0.0160 - val_mean_absolute_error: 0.0492
Epoch 2/3000
600000/600000 [==============================] - 0s 1us/sample - loss: 0.0150 - mean_absolute_error: 0.0371 - val_loss: 0.0141 - val_mean_absolute_error: 0.0334
Epoch 3/3000
600000/600000 [==============================] - 0s 1us/sample - loss: 0.0137 - mean_absolute_error: 0.0305 - val_loss: 0.0129 - val_mean_absolute_error: 0.0299
Epoch 4/3000
600000/600000 [==============================] - 0s 1us/sample - loss: 0.0125 - mean_absolute_error: 0.0289 - val_loss: 0.0116 - val_mean_absolute_error: 0.0269
Epoch 5/3000
600000/600000 [==============================] - 0s 1us/sample

Epoch 45/3000
600000/600000 [==============================] - 0s 1us/sample - loss: 0.0012 - mean_absolute_error: 0.0062 - val_loss: 0.0012 - val_mean_absolute_error: 0.0063
Epoch 46/3000
600000/600000 [==============================] - 0s 1us/sample - loss: 0.0011 - mean_absolute_error: 0.0061 - val_loss: 0.0011 - val_mean_absolute_error: 0.0056
Epoch 47/3000
600000/600000 [==============================] - 0s 1us/sample - loss: 0.0011 - mean_absolute_error: 0.0059 - val_loss: 0.0011 - val_mean_absolute_error: 0.0050
Epoch 48/3000
600000/600000 [==============================] - 0s 1us/sample - loss: 0.0010 - mean_absolute_error: 0.0059 - val_loss: 0.0010 - val_mean_absolute_error: 0.0053
Epoch 49/3000
600000/600000 [==============================] - 0s 1us/sample - loss: 0.0010 - mean_absolute_error: 0.0058 - val_loss: 0.0010 - val_mean_absolute_error: 0.0049
Epoch 50/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 9.7473e-04 - mean_absolute_error: 0.0056

Epoch 90/3000
600000/600000 [==============================] - 0s 1us/sample - loss: 3.3292e-04 - mean_absolute_error: 0.0035 - val_loss: 3.3008e-04 - val_mean_absolute_error: 0.0034
Epoch 91/3000
600000/600000 [==============================] - 0s 1us/sample - loss: 3.2954e-04 - mean_absolute_error: 0.0034 - val_loss: 3.2693e-04 - val_mean_absolute_error: 0.0040
Epoch 92/3000
600000/600000 [==============================] - 0s 1us/sample - loss: 3.2705e-04 - mean_absolute_error: 0.0035 - val_loss: 3.2359e-04 - val_mean_absolute_error: 0.0038
Epoch 93/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 3.2369e-04 - mean_absolute_error: 0.0033 - val_loss: 3.2066e-04 - val_mean_absolute_error: 0.0034
Epoch 94/3000
600000/600000 [==============================] - 0s 1us/sample - loss: 3.2102e-04 - mean_absolute_error: 0.0034 - val_loss: 3.1671e-04 - val_mean_absolute_error: 0.0032
Epoch 95/3000
600000/600000 [==============================] - 0s 1us/sample - loss: 

600000/600000 [==============================] - 1s 1us/sample - loss: 1.8708e-04 - mean_absolute_error: 0.0023 - val_loss: 1.8461e-04 - val_mean_absolute_error: 0.0024
Epoch 179/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8609e-04 - mean_absolute_error: 0.0022 - val_loss: 1.8363e-04 - val_mean_absolute_error: 0.0020
Epoch 180/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8515e-04 - mean_absolute_error: 0.0023 - val_loss: 1.8318e-04 - val_mean_absolute_error: 0.0020
Epoch 181/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8411e-04 - mean_absolute_error: 0.0022 - val_loss: 1.8127e-04 - val_mean_absolute_error: 0.0019
Epoch 182/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8343e-04 - mean_absolute_error: 0.0023 - val_loss: 1.8060e-04 - val_mean_absolute_error: 0.0018
Epoch 183/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8212e-0

600000/600000 [==============================] - 1s 1us/sample - loss: 1.2286e-04 - mean_absolute_error: 0.0019 - val_loss: 1.2212e-04 - val_mean_absolute_error: 0.0017
Epoch 267/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.2193e-04 - mean_absolute_error: 0.0017 - val_loss: 1.2147e-04 - val_mean_absolute_error: 0.0019
Epoch 268/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.2149e-04 - mean_absolute_error: 0.0018 - val_loss: 1.2104e-04 - val_mean_absolute_error: 0.0021
Epoch 269/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.2089e-04 - mean_absolute_error: 0.0018 - val_loss: 1.2013e-04 - val_mean_absolute_error: 0.0019
Epoch 270/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.2056e-04 - mean_absolute_error: 0.0019 - val_loss: 1.2017e-04 - val_mean_absolute_error: 0.0018
Epoch 271/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.1968e-0

...

600000/600000 [==============================] - 1s 1us/sample - loss: 1.9683e-05 - mean_absolute_error: 8.9634e-04 - val_loss: 1.9406e-05 - val_mean_absolute_error: 9.3951e-04
Epoch 834/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.9727e-05 - mean_absolute_error: 9.1046e-04 - val_loss: 1.9178e-05 - val_mean_absolute_error: 6.0909e-04
Epoch 835/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.9556e-05 - mean_absolute_error: 8.3800e-04 - val_loss: 1.9376e-05 - val_mean_absolute_error: 9.0580e-04
Epoch 836/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.9657e-05 - mean_absolute_error: 9.1210e-04 - val_loss: 2.0048e-05 - val_mean_absolute_error: 0.0013
Epoch 837/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.9658e-05 - mean_absolute_error: 9.2044e-04 - val_loss: 1.9958e-05 - val_mean_absolute_error: 0.0014
Epoch 838/3000
600000/600000 [==============================] -

600000/600000 [==============================] - 1s 1us/sample - loss: 1.9056e-05 - mean_absolute_error: 8.6920e-04 - val_loss: 1.8605e-05 - val_mean_absolute_error: 7.2645e-04
Epoch 877/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.9011e-05 - mean_absolute_error: 8.5460e-04 - val_loss: 1.8525e-05 - val_mean_absolute_error: 5.9119e-04
Epoch 878/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.9055e-05 - mean_absolute_error: 8.8374e-04 - val_loss: 1.8623e-05 - val_mean_absolute_error: 7.8831e-04
Epoch 879/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.9089e-05 - mean_absolute_error: 9.1194e-04 - val_loss: 1.8490e-05 - val_mean_absolute_error: 5.3267e-04
Epoch 880/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8991e-05 - mean_absolute_error: 8.7207e-04 - val_loss: 1.8512e-05 - val_mean_absolute_error: 6.5163e-04
Epoch 881/3000
600000/600000 [=========================

600000/600000 [==============================] - 1s 1us/sample - loss: 1.8605e-05 - mean_absolute_error: 9.1874e-04 - val_loss: 1.8065e-05 - val_mean_absolute_error: 6.9000e-04
Epoch 920/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8500e-05 - mean_absolute_error: 8.7136e-04 - val_loss: 1.7958e-05 - val_mean_absolute_error: 5.2308e-04
Epoch 921/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8544e-05 - mean_absolute_error: 9.0068e-04 - val_loss: 1.8072e-05 - val_mean_absolute_error: 7.6933e-04
Epoch 922/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8538e-05 - mean_absolute_error: 9.0085e-04 - val_loss: 1.8085e-05 - val_mean_absolute_error: 5.6233e-04
Epoch 923/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8581e-05 - mean_absolute_error: 9.2185e-04 - val_loss: 1.8020e-05 - val_mean_absolute_error: 6.3364e-04
Epoch 924/3000
600000/600000 [=========================

600000/600000 [==============================] - 1s 1us/sample - loss: 1.8098e-05 - mean_absolute_error: 8.7603e-04 - val_loss: 1.7516e-05 - val_mean_absolute_error: 5.8791e-04
Epoch 963/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8159e-05 - mean_absolute_error: 9.3274e-04 - val_loss: 1.7504e-05 - val_mean_absolute_error: 5.7837e-04
Epoch 964/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8015e-05 - mean_absolute_error: 8.4561e-04 - val_loss: 1.7652e-05 - val_mean_absolute_error: 7.3226e-04
Epoch 965/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8123e-05 - mean_absolute_error: 9.2248e-04 - val_loss: 1.7840e-05 - val_mean_absolute_error: 9.8775e-04
Epoch 966/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.8040e-05 - mean_absolute_error: 8.6812e-04 - val_loss: 1.8997e-05 - val_mean_absolute_error: 0.0016
Epoch 967/3000
600000/600000 [=============================

600000/600000 [==============================] - 1s 1us/sample - loss: 1.7758e-05 - mean_absolute_error: 9.1378e-04 - val_loss: 1.7734e-05 - val_mean_absolute_error: 0.0012
Epoch 1006/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.7661e-05 - mean_absolute_error: 8.6576e-04 - val_loss: 1.7133e-05 - val_mean_absolute_error: 6.5655e-04
Epoch 1007/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.7617e-05 - mean_absolute_error: 8.4827e-04 - val_loss: 1.7836e-05 - val_mean_absolute_error: 0.0013
Epoch 1008/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.7715e-05 - mean_absolute_error: 9.0280e-04 - val_loss: 1.7440e-05 - val_mean_absolute_error: 9.7288e-04
Epoch 1009/3000
600000/600000 [==============================] - 1s 1us/sample - loss: 1.7659e-05 - mean_absolute_error: 8.6324e-04 - val_loss: 1.7406e-05 - val_mean_absolute_error: 9.8253e-04
Epoch 1010/3000
600000/600000 [============================

In [14]:
# Save the model
model.save('plastic_strain_predictor_model_von_mises_10_01_2025_10_0.0001.h5')
#print("Model saved as 'plastic_strain_predictor_model.h5'")

with open('scaler_X_von_mises_10_01_10_0.0001.pkl', 'wb') as file:
    pickle.dump(scaler_X, file)
with open('scaler_y_von_mises_10_01_10_0.0001.pkl', 'wb') as file:
    pickle.dump(scaler_y, file)

In [15]:
training[:, :4]

array([[ 1.92940197e+11,  1.18162023e+08,  1.09658630e+01,
        -8.56051710e+07],
       [ 1.92940197e+11,  1.18162023e+08,  1.09658630e+01,
        -6.58384778e+07],
       [ 1.92940197e+11,  1.18162023e+08,  1.09658630e+01,
        -4.06378382e+07],
       ...,
       [ 1.93033797e+11,  1.22540732e+08,  5.91406038e+00,
         2.46520491e+07],
       [ 1.93033797e+11,  1.22540732e+08,  5.91406038e+00,
         3.12384367e+07],
       [ 1.93033797e+11,  1.22540732e+08,  5.91406038e+00,
         2.49608189e+07]])